Reading Files for Neighboring Census Blocks

In [2]:
!pip3 install geopandas

     |████████████████████████████████| 1.0 MB 4.3 MB/s 
     |████████████████████████████████| 15.4 MB 37 kB/s 
     |████████████████████████████████| 6.3 MB 28.3 MB/s 


In [ ]:
import geopandas as gpd
file = gpd.read_file("mi_pl2020_b.shp")
file.to_file("marFile.json", driver="GeoJSON")

In [3]:
!pip install turfpy

  Created wheel for turfpy: filename=turfpy-0.0.7-py3-none-any.whl size=39119 sha256=ce2e1ebfe004a205d16fb1e42ef8216ecafd84837ad4d03d0aadc7753192feca
  Stored in directory: /root/.cache/pip/wheels/56/d0/d8/13858feab5280185516cd22df1adc0f75426968815b057dc5e
Successfully built turfpy


In [6]:
from turfpy import measurement
from geojson import Point, Feature
from turfpy.transformation import intersect
from turfpy.measurement import length
import json

In [ ]:
file = gpd.read_file("mi_pl2020_b.shp")

In [ ]:
file.to_file("marFile.json", driver="GeoJSON")

In [ ]:
import json

Reading Census Block File

In [ ]:
f = open('marylandFile.json')
data = json.load(f)

precinctList = {}
censusBlockList = []

uniquePrecincts = set()
uniqueCongressionalDistricts = set()

allFields = {}

cBNum = 0;

for cb in data['features']:

  for attribute, value in cb['properties'].items():
    if attribute not in allFields:
      allFields[attribute] = set()
  
    allFields[attribute].add(value)
  
  uniquePrecincts.add(cb['properties']['VTD'])
  uniqueCongressionalDistricts.add(cb['properties']['CD116'])

  censusBlock = {}
  censusBlock['CBID'] = cBNum

  censusBlock['VTD'] = cb['properties']['VTD']
  censusBlock['CD'] = cb['properties']['CD116']
  censusBlock['NAME'] = cb['properties']['BASENAME']
  censusBlock['INT'] = [cb['properties']['INTPTLAT'], cb['properties']['INTPTLON']]
  
  censusBlock['TOTALPOP'] = cb['properties']['P0020001']
  censusBlock['WHITE'] = cb['properties']['P0020005']
  censusBlock['BLACK'] = cb['properties']['P0020006']
  censusBlock['HISPANIC'] = cb['properties']['P0020002']
  censusBlock['AMERICANINDIAN'] = cb['properties']['P0020007']
  censusBlock['ASIAN'] = cb['properties']['P0020008']
  censusBlock['HAWAIIAN'] = cb['properties']['P0020009']
  censusBlock['OTHER'] = cb['properties']['P0020010']

  censusBlock['GEOMETRY'] = cb['geometry']['coordinates']

  censusBlock['ADJACENT_CBS'] = []

  cBNum += 1

  if cb['properties']['VTD'] in precinctList:
    precinctList[cb['properties']['VTD']].append(censusBlock)
  else:
    precinctList[cb['properties']['VTD']] = [censusBlock]
  
  censusBlockList.append(censusBlock)
 
# with open('marylandFilePostProcessing.json', 'w') as f:
#   json.dump(precinctList, f, ensure_ascii=False)

In [ ]:
for key, value in allFields.items():
  if len(value) > 1420:
    print(key, len(value))



#print(data['features'][0])

TRACTCE20 1428
GEOID20 83827
ALAND20 56670
AWATER20 9142
INTPTLAT20 83509
INTPTLON20 83607
LOGRECNO 83827
GEOID 83827
GEOCODE 83827
TRACT 1428
AREALAND 56670
AREAWATR 9142
INTPTLAT 83509
INTPTLON 83607


Create Census Block Adjacency List

In [ ]:
def returnCBFeature(cb):
    return Feature(geometry={'coordinates': cb['GEOMETRY'], 'type': 'Polygon'})
    
for cb in censusBlockList:
  cb['FEATURE'] = returnCBFeature(cb)

NameError: ignored

In [ ]:
# Census Block Adjacency List
for cB in censusBlockList:
  for cB2 in censusBlockList:
    if cB['CBID'] != cB2['CBID']:
      if(len(cB['GEOMETRY'][0]) >= 3 and len(cB2['GEOMETRY'][0]) >= 3):
        it = intersect([cB['FEATURE'], cB2['FEATURE']])
        if it:
          boundaryLen = length(it, units="ft")
          if boundaryLen >= 100:
            cB['ADJACENT_CBS'].append(cB2['CBID'])

Test (Intermission)

In [ ]:
v = open('virginia.geojson')
wv = open('west virginia.geojson')
vdata = json.load(v)
wvdata = json.load(wv)
vGeometry = vdata['geometry']
wvGeometry = wvdata['geometry']
it = intersect([vGeometry, wvGeometry])
print(length(it, units="ft"))

2184250.8607852412


Reading Precinct for Neighboring Precincts

In [ ]:
mfd = open('michiganPrecincts.json')
mfdData = json.load(mfd)

In [ ]:
precincts = [] # NOT precinctList

In [ ]:
for p in mfdData['features']:

  precinct = {}

  if p['properties']['NAME20'] == 'Voting Districts Not Defined':
    continue
  precinct['NAME'] = p['properties']['NAME20']
  
  precinct['TOTALPOP'] = p['properties']['P0020001']
  precinct['WHITE'] = p['properties']['P0020005']
  precinct['BLACK'] = p['properties']['P0020006']
  precinct['HISPANIC'] = p['properties']['P0020002']
  precinct['AMERICANINDIAN'] = p['properties']['P0020007']
  precinct['ASIAN'] = p['properties']['P0020008']
  precinct['HAWAIIAN'] = p['properties']['P0020009']
  precinct['OTHER'] = p['properties']['P0020010']

  precinct['TYPE'] = p['geometry']['type']
  precinct['GEOMETRY'] = p['geometry']['coordinates']

  precinct['ADJACENT_PRECINCTS'] = []
  
  precincts.append(precinct)

In [ ]:
print(precincts[100])

{'NAME': '1632200005449', 'TOTALPOP': 1127, 'WHITE': 55, 'BLACK': 1014, 'HISPANIC': 33, 'AMERICANINDIAN': 0, 'ASIAN': 3, 'HAWAIIAN': 0, 'OTHER': 2, 'TYPE': 'Polygon', 'GEOMETRY': [[[-83.11390399999999, 42.364657], [-83.116207, 42.363794999999996], [-83.118344, 42.362994], [-83.121027, 42.36199], [-83.12124, 42.361695999999995], [-83.121301, 42.361596], [-83.120539, 42.361292], [-83.11992699999999, 42.361045999999995], [-83.11936299999999, 42.360819], [-83.117375, 42.361564], [-83.116874, 42.360824], [-83.117463, 42.360605], [-83.118162, 42.360335], [-83.117413, 42.360033], [-83.117019, 42.359873], [-83.11655999999999, 42.359687], [-83.116085, 42.359502], [-83.115867, 42.359407], [-83.1147, 42.358920999999995], [-83.113541, 42.358464999999995], [-83.11239499999999, 42.358005], [-83.112246, 42.357943999999996], [-83.11092599999999, 42.35741], [-83.11058, 42.35727], [-83.10955899999999, 42.356857], [-83.10925499999999, 42.356736], [-83.108716, 42.356930999999996], [-83.105386, 42.358174],

Create Precinct Adjacency List

In [ ]:
# generating features is hard and doing it n^2 times is stupid 
# generate all the features beforehand, then compare them

def returnFeature(p):
  if p['TYPE'] == 'Polygon':
    return Feature(geometry={'coordinates': p['GEOMETRY'], 'type': 'Polygon'})
  else:
    return Feature(geometry={'coordinates': p['GEOMETRY'], 'type': 'MultiPolygon'})

for p in precincts:
  p['FEATURE'] = returnFeature(p)

In [ ]:
# Precinct Adjacency List
precinctLen = len(precincts)
for i1 in range(precinctLen-1):
  for i2 in range(i1+1, precinctLen):
    p1 = precincts[i1]
    p2 = precincts[i2]
    if(len(p1['GEOMETRY'][0]) >= 3 and len(p2['GEOMETRY'][0]) >= 3):
      it = intersect([p1['FEATURE'], p2['FEATURE']])
      if it:
        boundaryLen = length(it, units="ft")
        if boundaryLen >= 200:
          p1['ADJACENT_PRECINCTS'].append(p2['NAME'])
          p2['ADJACENT_PRECINCTS'].append(p1['NAME'])

KeyboardInterrupt: ignored

In [ ]:
print(len(precincts))

9530


In [ ]:
# Generating Adjacent Precincts
precinctAdjacencyList = {}
for p in precincts:
  precinctAdjacencyList[p['NAME']] = p['ADJACENT_PRECINCTS']
print(precinctAdjacencyList)

{'0030446000001': [], '0035622000001': ['0035620000001', '0033338000001', 'Voting Districts Not Defined', '0031196000001', '0415650000001', '1534084000001', '1533794000001'], '0036916000001': ['0036082000001', '0034754000001', '1037411000001'], '0035236000001': ['0034754000001', '0415650000001', '0415222000001'], '0036082000001': ['0036916000001', 'Voting Districts Not Defined', '1037411000001', '1031566000002', 'Voting Districts Not Defined'], '0035620000001': ['0035622000001', '0033338000001'], '0033338000001': ['0035622000001', '0035620000001', 'Voting Districts Not Defined'], 'Voting Districts Not Defined': [], '1298560000001': ['1293932000001', '1291574000001', '1294364000001', '1298558001001'], '1292986000001': ['1296960000001', '1294364000001', '1350822000001', '1436820000002', '1430432000001'], '1295434000001': ['1293932000001', '1296832000001', '1291574000001', '1295434000002'], '1296030000001': [], '1293286000001': ['1296960000001', '1293830000001', '1355312000001', '06964640

In [ ]:
output_file = open('michiganAdjacencyPrecinctsNEW.json', 'w')
json.dump(precinctAdjacencyList, output_file, indent=6)

In [15]:
precincts = json.load(open('marylandPrecincts.json'))
boundaryDict = {}

# Precinct Adjacency List
for p1 in precincts['features']:
  if(p1['properties']['GEOID20'] == '2401301-001'):
    print('YEET')
    for p2 in precincts['features']:
      if(p2['properties']['GEOID20'] == '2401303-001'):
        print('YEET')
        p1F = Feature(geometry=p1['geometry'])
        p2F = Feature(geometry=p2['geometry'])
        it = intersect([p1F, p2F])
        print('YEET')
        if it:
          boundaryLen = length(it, units="m")
          print(boundaryLen)
          # boundaryDict[p1['properties']['GEOID20'] + '->' + p2['properties']['GEOID20']] = boundaryLen
    # if(len(p1['GEOMETRY'][0]) >= 3 and len(p2['GEOMETRY'][0]) >= 3):
    #   it = intersect([p1F, p2F])
    #   if it:
    #     boundaryLen = length(it, units="ft")
    #     if boundaryLen >= 200:
    #       p1['ADJACENT_PRECINCTS'].append(p2['NAME'])
    #       p2['ADJACENT_PRECINCTS'].append(p1['NAME'])

YEET
YEET
YEET
3116.004644361358
